In [53]:
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import re
import scipy.stats as ss
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm.sklearn import LGBMClassifier

In [66]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
PassengerId = test['PassengerId']
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [67]:
full_data = [train, test]

In [68]:
# Gives the length of the name
train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)
# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
# Remove all NULLS in the Fare column and create a new feature CategoricalFare
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)

# Create a New feature CategoricalAge
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
train['CategoricalAge'] = pd.cut(train['Age'], 5)

# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""
# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4 ;

In [69]:
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)
test  = test.drop(drop_elements, axis = 1)
y = train['Survived']
X_train = train.drop(["Survived"], axis=1)
X_test = test
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.1, random_state=123)

In [70]:
def xgb_model(X_train, y_train, X_val, y_val, save_file, folds, param_comb, n_jobs, scoring):
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 123)
    xgb = XGBClassifier(n_jobs=n_jobs, random_state=123)
    params = {"learning_rate":ss.uniform(0.001,0.5),
             "n_estimators":[1000],
             "max_depth": ss.randint(3,10),
             "min_child_weight":ss.randint(1,15),
             "gamma": ss.randint(0,10),
             "subsample": [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
             "colsample_bytree": [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
             "reg_lambda":ss.uniform(0.05,5),
             "reg_alpha":ss.uniform(0.05,5)}
    model_xgb =  RandomizedSearchCV(estimator=xgb, 
                                    param_distributions=params, 
                                    n_iter=param_comb, 
                                    scoring=scoring, 
                                    n_jobs=n_jobs, 
                                    cv=skf.split(X_train, y_train), 
                                    verbose=0, 
                                    random_state=123)
    model_xgb.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='accuracy', early_stopping_rounds=50)
    joblib.dump(model_xgb.best_estimator_, f'best_{save_file}', compress = 1)
    return model_xgb


def rf_model(X_train, y_train, save_file, folds, param_comb, n_jobs, scoring):
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 123)
    rf = RandomForestClassifier(n_jobs=n_jobs, random_state=123)
    params = {"bootstrap": [True, False],
             "max_depth": ss.randint(3,10),
             "max_features": ['auto', 'sqrt'],
             "min_samples_leaf": ss.randint(1,10),
             "min_samples_split": ss.randint(2,10),
             "n_estimators":[1000],
             "criterion": ["gini", "entropy"],
             "class_weight": ["balanced", "balanced_subsample", None]
             }
    model_rf =  RandomizedSearchCV(estimator=rf, 
                                    param_distributions=params, 
                                    n_iter=param_comb, 
                                    scoring=scoring, 
                                    n_jobs=n_jobs, 
                                    cv=skf.split(X_train, y_train), 
                                    verbose=0, 
                                    random_state=123)
    model_rf.fit(X_train, y_train)
    joblib.dump(model_rf.best_estimator_, f'best_{save_file}', compress = 1)
    return model_rf


def lgb_model(X_train, y_train, X_val, y_val, save_file, folds, param_comb, n_jobs, scoring):
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 123)
    lgb = LGBMClassifier(n_jobs=n_jobs, random_state=123)
    params = {"num_leaves":ss.randint(2,50),
              "max_depth": ss.randint(3,10),
              "learning_rate":ss.uniform(0.001,0.5),
              "n_estimators":[1000],
              "objective": ["binary"],
              "class_weight": ["balanced", None],
              "subsample": [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              "colsample_bytree": [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              "reg_lambda":ss.uniform(0.05,5),
              "reg_alpha":ss.uniform(0.05,5),
              "min_child_weight":ss.randint(1,15)}
    model_lgb =  RandomizedSearchCV(estimator=lgb, 
                                    param_distributions=params, 
                                    n_iter=param_comb, 
                                    scoring=scoring, 
                                    n_jobs=n_jobs, 
                                    cv=skf.split(X_train, y_train), 
                                    verbose=0, 
                                    random_state=123)
    model_lgb.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='accuracy', early_stopping_rounds=50)
    joblib.dump(model_lgb.best_estimator_, f'best_{save_file}', compress = 1)
    return model_lgb

folds = 10
param_comb = 100
n_jobs = -1
scoring = "accuracy"

model_xgb = xgb_model(X_train, y_train, X_val, y_val, 'xgb.pkl', folds, param_comb, n_jobs, scoring)
model_rf = rf_model(X_train, y_train, 'rf.pkl', folds, param_comb,  n_jobs, scoring)
model_lgb = lgb_model(X_train, y_train, X_val, y_val, 'lgb.pkl', folds, param_comb,  n_jobs, scoring)

# ### Optional
# model_xgb = joblib.load('xgb.sav')
# model_rf = joblib.load('rf.sav')
# model_lgb = joblib.load('lgb.sav')

XGB_predictions = model_xgb.predict(X_test)
RF_predictions = model_rf.predict(X_test)
LGB_predictions = model_lgb.predict(X_test)
ensemble_prediction =  XGB_predictions + RF_predictions + LGB_predictions

predictions = {"xgb": XGB_predictions, 
               "rf": RF_predictions, 
               "lgb": LGB_predictions, 
               "ensemble": ensemble_prediction}

for pred in predictions:
    df = pd.DataFrame({"PassengerId": PassengerId, 
                      "Survived": predictions[pred]})
    if pred == "ensemble":
        df["Survived"] = df["Survived"].apply(lambda x: 0 if x < 2 else 1)
    df.to_csv(f'submissions/{pred}_submit.csv', index=None)

JoblibTypeError: JoblibTypeError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/adam/anaconda3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/home/adam/anaconda3/lib/python3.